This is based on Fabian Pedregosas [blog](https://fa.bianp.net/blog/2013/logistic-ordinal-regression/) and deprecated [github](https://github.com/fabianp/minirank/blob/master/minirank/logistic.py)

In [5]:
import numpy as np
from numpy.random import default_rng
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from scipy import optimize, linalg, sparse
from matplotlib import pyplot as plt

In [6]:
rng = default_rng(1234)

# Simulation Data

Use a subsample of the simulation data for testing


In [7]:
from masterthesis.data import load_h5ad, load_acinar
data = load_h5ad("/home/julian/Uni/MasterThesis/data/simdata.h5ad")

In [8]:
idx = data.var['Setting'] == "TS"  # extract only the time series samples
sim_X = data.X[:, idx]
sim_y = data.obs["Ordinal_Time_Labels"]

In [9]:
# random subsample genes
y_idx = rng.choice(np.arange(sim_y.size), size=sim_y.size // 2, replace=False)
x_idx = rng.choice(np.arange(sim_X.shape[1]), size=sim_X.shape[1] // 4, replace=False)
sim_X = sim_X[y_idx, :]
sim_X = sim_X[:, x_idx]
sim_y = sim_y[y_idx]

In [10]:
scaler = StandardScaler()
sim_X = scaler.fit_transform(sim_X)

In [11]:
print(sim_X.shape)
print(sim_y.shape)

(286, 1031)
(286,)


# Acinar Data

In [14]:
acinar_ann = load_h5ad("/home/julian/Uni/MasterThesis/data/acinar_sce.h5ad")

In [94]:
# selected Genes after preprocessing in R
sel_genes = ["REG3A", "AMY2A", "MT2A", "OLFM4",
             "SYCN", "CELA2B", "FGL1", "AMY2B",
             "MT1G", "TM4SF1", "CELA2A", "PDK4", 
             "TACSTD2", "CD44", "PNLIPRP2", "ALB", 
             "ERP27", "LDHA", "REG3G", "CTRL", "CLPS",
             "FOS", "HSPA8", "SERPINA3", "CELA3B", "CRP"]
sel_genes = sorted(sel_genes)

In [95]:
from sklearn.model_selection import train_test_split

ac_y = np.array([int(x) for x in acinar_ann.obs.donor_age])
ac_label_conv = dict(zip(np.unique(ac_y), range(len(ac_y))))
ac_y = np.array([ac_label_conv[l] for l in ac_y])
k = len(np.unique(ac_y))

ac_X_train, ac_X_test, ac_y_train, ac_y_test = train_test_split(acinar_ann[:,sel_genes].X, ac_y, 
                                                                test_size=0.1, 
                                                                stratify=ac_y,
                                                                random_state=1234)

In [96]:
labels = np.sort(np.unique(ac_y))
conversion = dict(zip(labels, np.arange(len(labels)))) 
ac_y_train_trans = np.array([conversion[e] for e in ac_y_train])
ac_y_test_trans = np.array([conversion[e] for e in ac_y_test])

In [97]:
scaler = StandardScaler()
ac_X_train = scaler.fit_transform(ac_X_train)
ac_X_test = scaler.fit_transform(ac_X_test)

## Gradient DIY

In [98]:
# Mord solution with loss changed to L1
# https://github.com/fabianp/mord/blob/master/mord/threshold_based.py
import numpy as np
from sklearn import base, metrics
from sklearn.utils.validation import check_X_y
from scipy import optimize

def sigmoid(t):
    # sigmoid function, 1 / (1 + exp(-t))
    # stable computation
    idx = t > 0
    out = np.zeros_like(t)
    out[idx] = 1. / (1 + np.exp(-t[idx]))
    exp_t = np.exp(t[~idx])
    out[~idx] = exp_t / (1. + exp_t)
    return out


def log_loss(Z):
    # stable computation of the logistic loss
    idx = Z > 0
    out = np.zeros_like(Z)
    out[idx] = np.log(1 + np.exp(-Z[idx]))
    out[~idx] = (-Z[~idx] + np.log(1 + np.exp(Z[~idx])))
    return out


def obj_margin(x0, X, y, alpha, n_class, weights, L, sample_weight):
    """
    Objective function for the general margin-based formulation
    """

    w = x0[:X.shape[1]]
    c = x0[X.shape[1]:]
    theta = L.dot(c)
    loss_fd = weights[y]

    Xw = X.dot(w)
    Alpha = theta[:, None] - Xw  # (n_class - 1, n_samples)
    S = np.sign(np.arange(n_class - 1)[:, None] - y + 0.5)

    err = loss_fd.T * log_loss(S * Alpha)
    if sample_weight is not None:
        err *= sample_weight
    obj = np.sum(err)
    
    # changed to L1 regularization
    obj += alpha * np.sum(np.abs(w))
    return obj


def grad_margin(x0, X, y, alpha, n_class, weights, L, sample_weight):
    """
    Gradient for the general margin-based formulation
    """

    w = x0[:X.shape[1]]
    c = x0[X.shape[1]:]
    theta = L.dot(c)
    loss_fd = weights[y]

    Xw = X.dot(w)
    Alpha = theta[:, None] - Xw  # (n_class - 1, n_samples)
    S = np.sign(np.arange(n_class - 1)[:, None] - y + 0.5)
    # Alpha[idx] *= -1
    # W[idx.T] *= -1

    Sigma = S * loss_fd.T * sigmoid(-S * Alpha)
    if sample_weight is not None:
        Sigma *= sample_weight

    grad_w = X.T.dot(Sigma.sum(0)) + alpha * np.sign(w)

    grad_theta = -Sigma.sum(1)
    grad_c = L.T.dot(grad_theta)
    return np.concatenate((grad_w, grad_c), axis=0)


def threshold_fit(X, y, alpha, n_class, mode='AE',
                  max_iter=1000, verbose=False, tol=1e-12,
                  sample_weight=None):
    """
    Solve the general threshold-based ordinal regression model
    using the logistic loss as surrogate of the 0-1 loss

    Parameters
    ----------
    mode : string, one of {'AE', '0-1', 'SE'}

    """

    X, y = check_X_y(X, y, accept_sparse='csr')
    unique_y = np.sort(np.unique(y))
    if not np.all(unique_y == np.arange(unique_y.size)):
        raise ValueError(
            'Values in y must be %s, instead got %s'
            % (np.arange(unique_y.size), unique_y))

    n_samples, n_features = X.shape

    # convert from c to theta
    L = np.zeros((n_class - 1, n_class - 1))
    L[np.tril_indices(n_class-1)] = 1.

    if mode == 'AE':
        # loss forward difference
        loss_fd = np.ones((n_class, n_class - 1))
    elif mode == '0-1':
        loss_fd = np.diag(np.ones(n_class - 1)) + \
            np.diag(np.ones(n_class - 2), k=-1)
        loss_fd = np.vstack((loss_fd, np.zeros(n_class - 1)))
        loss_fd[-1, -1] = 1  # border case
    elif mode == 'SE':
        a = np.arange(n_class-1)
        b = np.arange(n_class)
        loss_fd = np.abs((a - b[:, None])**2 - (a - b[:, None]+1)**2)
    else:
        raise NotImplementedError

    x0 = np.zeros(n_features + n_class - 1)
    x0[X.shape[1]:] = np.arange(n_class - 1)
    options = {'maxiter' : max_iter, 'disp': verbose}
    if n_class > 2:
        bounds = [(None, None)] * (n_features + 1) + \
                 [(0, None)] * (n_class - 2)
    else:
        bounds = None
    
    #sol_method = 'L-BFGS-B'
    #sol_method = 'Newton-CG'
    sol_method = 'SLSQP'
    sol = optimize.minimize(obj_margin, x0, method=sol_method,
        jac=grad_margin, bounds=bounds, options=options,
        args=(X, y, alpha, n_class, loss_fd, L, sample_weight),
        tol=tol)
    if verbose and not sol.success:
        print(sol.message)

    w, c = sol.x[:X.shape[1]], sol.x[X.shape[1]:]
    theta = L.dot(c)
    return w, theta


def threshold_predict(X, w, theta):
    """
    Class numbers are assumed to be between 0 and k-1
    """
    tmp = theta[:, None] - np.asarray(X.dot(w))
    pred = np.sum(tmp < 0, axis=0).astype(int)
    return pred


def threshold_proba(X, w, theta):
    """
    Class numbers are assumed to be between 0 and k-1. Assumes
    the `sigmoid` link function is used.
    """
    eta = theta[:, None] - np.asarray(X.dot(w), dtype=np.float64)
    prob = np.pad(
        sigmoid(eta).T,
        pad_width=((0, 0), (1, 1)),
        mode='constant',
        constant_values=(0, 1))
    return np.diff(prob)



class LogisticIT(base.BaseEstimator):
    """
    Classifier that implements the ordinal logistic model
    (Immediate-Threshold variant).

    Contrary to the OrdinalLogistic model, this variant
    minimizes a convex surrogate of the 0-1 loss, hence
    the score associated with this object is the accuracy
    score, i.e. the same score used in multiclass
    classification methods (sklearn.metrics.accuracy_score).

    Parameters
    ----------
    alpha: float
        Regularization parameter. Zero is no regularization, higher values
        increate the squared l2 regularization.

    References
    ----------
    J. D. M. Rennie and N. Srebro, "Loss Functions for Preference Levels :
    Regression with Discrete Ordered Labels," in Proceedings of the IJCAI
    Multidisciplinary Workshop on Advances in Preference Handling, 2005.
    """
    def __init__(self, alpha=1., verbose=0, max_iter=1000):
        self.alpha = alpha
        self.verbose = verbose
        self.max_iter = max_iter

    def fit(self, X, y, sample_weight=None):
        _y = np.array(y).astype(int)
        if np.abs(_y - y).sum() > 0.1:
            raise ValueError('y must only contain integer values')
        self.classes_ = np.unique(y)
        self.n_class_ = self.classes_.max() - self.classes_.min() + 1
        y_tmp = y - y.min()  # we need classes that start at zero
        self.coef_, self.theta_ = threshold_fit(
            X, y_tmp, self.alpha, self.n_class_,
            mode='0-1', verbose=self.verbose, max_iter=self.max_iter,
            sample_weight=sample_weight)
        return self

    def predict(self, X):
        return threshold_predict(X, self.coef_, self.theta_) +\
         self.classes_.min()

    def predict_proba(self, X):
        return threshold_proba(X, self.coef_, self.theta_)

    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        return metrics.accuracy_score(
            pred,
            y,
            sample_weight=sample_weight)


In [101]:
regressor = LogisticIT(verbose=0, alpha=10)
regressor.fit(ac_X_train, ac_y_train_trans)
regressor.score(ac_X_test, ac_y_test_trans)

0.38095238095238093

In [104]:
#regressor.coef_[regressor.coef_ < 0.0001] = 0
print(metrics.balanced_accuracy_score(ac_y_test_trans, regressor.predict(ac_X_test)))
print(regressor.coef_)
print(regressor.theta_)

0.26988636363636365
[ 1.23778430e-01  2.08324304e-01  1.76691039e-01  2.98701886e-01
  5.02328584e-10 -8.90480109e-02  1.45208842e-06 -1.90163349e-01
  5.31889679e-06 -1.13353012e-01 -1.58525750e-01  7.85131379e-04
 -1.28584612e-01 -3.67297772e-02  4.25454457e-06  1.40584411e-05
  2.18059561e-01 -8.60156434e-06  2.75518533e-05 -1.21339646e-05
  3.14973061e-01 -1.85079684e-01  2.32742630e-01  5.45999271e-06
  1.36101249e-05  1.24626790e-01]
[-1.76577103  0.5383947   0.5383947   0.59014072  0.59014072  1.20320139
  1.20320139]


Gradient based on derivatives and Fabianp's code

In [13]:
BIG = 1e10
SMALL = 1e-6

def phi(beta, thresholds, X):
    phi = 1 / (1 + np.exp(beta.T @ X - thresholds))
    return phi

def l1_regularizer(beta, l):
    return l * np.sum(np.abs(beta))

# PROBLEM: Is the objective minimized or maximized?? -> Scipy MINIMIZES
# PROBLEM: What if diff is negative? -> Log has issues, but value may be meaningful
# PROBLEM 2: What if diff is too small? -> Log will explode
# PROBLEM 3: Loss of precision?
# log likelihood with regularization as objective function
def objective(params, X, y, k, lamb=0.1, scale_by_y=True):
    
    beta = params[:-k]
    theta = params[-k:]
    thresholds = np.array([theta[i] for i in y])
    thresholds_m1 = np.array([theta[max(0, i-1)] for i in y]) 

    # fit term
    idx = (thresholds > 0)
    diff = np.zeros_like(thresholds)
    phi_i = phi(beta, thresholds, X)
    phi_im1 = phi(beta, thresholds_m1, X)
    
    diff[idx] = (phi_i - phi_im1)[idx]
    diff[~idx] = phi_i[~idx]

    # cut off the difference at a minimum to avoid PROBLEM 2
    #if diff.min() < SMALL:
    #    diff = np.maximum(diff, SMALL)
    
    loglik = np.log(diff, out=np.zeros_like(diff), where=diff>0)
    
    loglik = np.sum(loglik)

    # regularization term
    loglik -= l1_regularizer(beta, lamb)
    
    # scale the loss with the inverse number of samples to handle PROBLEM 3
    if scale_by_y:
        loglik *= (1 / y.size)

    return loglik

def loss(*args, **kwargs):
    return - objective(*args, **kwargs)

def objective_grad(params, X, y, k, lamb=0.1, scale_by_y=True):
    beta = params[:-k]
    theta = params[-k:]
    thresholds = np.array([theta[i] for i in y])
    thresholds_m1 = np.array([theta[max(0, i-1)] for i in y])

    # PROBLEM: thresholds_m1 - thresholds can be 0 -> then the denominator becomes 0!
    idx = (thresholds > 0)
    diff = np.zeros_like(thresholds)
    phi_i = phi(beta, thresholds, X)
    phi_im1 = phi(beta, thresholds_m1, X)
    
    diff[idx] = (phi_i - phi_im1)[idx]
    diff[~idx] = phi_i[~idx]
    
    # BETA UPDATE
    beta_grad = np.sum(X * (1 - diff), axis=1)
    
    # derivative of regularizer over beta scaled
    beta_grad += np.sum(np.sign(beta) * (beta == 0)) * lamb
    
    # THETA UPDATE
    # first half of the gradient
    e = np.identity(k)
    e_expanded = np.concatenate([e[i] for i in y]).reshape(y.size, k).T
    
    temp = (1 
            - phi(beta, thresholds, X) 
            - np.exp(np.log(1 - (1 - np.exp(diff)))))
    t1 = e_expanded @ temp
    
    # second half of the gradient
    e_m1 = np.identity(k+1)[1:,:-1]  # identity with diagonal shifty up by one
    e_m1_expanded = np.concatenate([e_m1[i] for i in y]).reshape(y.size, k).T
    temp = (1 
            - phi(beta, thresholds_m1, X) 
            - np.exp(np.log(1 - (1 - np.exp(-1 * (diff))))))
    
    t2 = e_m1_expanded @ temp
    
    theta_grad = t1 + t2
    
    # scaling
    if scale_by_y:
        theta_grad = theta_grad * (1/theta.size)
        beta_grad = beta_grad * (1/beta.size)

    return np.concatenate([beta_grad, theta_grad])

In [14]:
from scipy.optimize import minimize

def train(obj, grad, X, y, lamb=0.9, scaling=False, method="BFGS"):
    # flip such that X -> (genes, cells)
    if X.shape[0] == y.size:
        X = X.T
    
    n_classes = np.unique(y).size
    params = np.zeros(X.shape[0] + n_classes)
    
    m = minimize(obj, params_nm, args=(X, y, n_classes, lamb, scaling), jac=grad, method=method)
    print(m)
    print(m.x)

In [15]:
train(loss, objective_grad, ac_X_train, ac_y_train, lamb=0., scaling=True, method="bfgs")

NameError: name 'params_nm' is not defined

In [ ]:
from scipy.optimize import minimize

scale_by_y = False
lamb = 0.9
n_classes = np.unique(ac_y_train).size
params = np.zeros(ac_X_train.shape[1] + n_classes)

# random params
params_rand = rng.normal(0, 0.1, ac_X_train.shape[1] + n_classes)

# params after one run of Nelder-Mead 
params_nm = [2.99552789e-17, -9.12238797e-17,  8.43373503e-17,  6.28923016e-17,
        1.29517064e-16,  2.35470728e-16,  4.64152560e-18, -2.37379072e-17,
        9.33068062e-17, -1.31213310e-16,  8.47381150e-17, -9.14277033e-18,
       -1.10647317e-16, -5.55542955e-17,  4.15674610e-17, -7.28795221e-17,
        1.25000000e-04, -3.91915815e-17,  1.01399459e-16,  2.16536932e-16,
        6.13107712e-15,  1.23078388e-16,  1.16260966e-16,  1.76600273e-17,
        1.35598036e-16,  7.26746491e-17,  1.25000000e-04,  1.30296841e-16,
        1.45441669e-16, -1.72678216e-17, -1.87164751e-16, -4.54465550e-17,
       -1.00882237e-16, -3.29592381e-17]

params_warm =  [0.47077975,  0.        ,  0.37275445,  0.        ,  0.        ,
        -0.2718176 ,  0.01834662,  0.27956828,  0.        ,  0.00906079,
         0.        ,  0.22041753, -0.1378634 ,  0.18726233, -0.08640451,
         0.08268969, -0.26427331,  0.        , -0.28873376, -0.24587408,
         0.        , -0.07656282, -0.02134246,  0.59662556,  0.05288458,
         0.        ,  1.71402649,  0.10353958,  0.        , 
        -0.48783945, -0.62012817, -1.75287372, -2.09708938, 4]

m = minimize(objective, params_nm, args=(ac_X_train.T, ac_y_train, n_classes, lamb, scale_by_y),
         method="BFGS")
print(m)
print(m.x)

### Observations:

- Numerical instabilities (nans, infs): Especially BFGS struggles here and usually only executes one iteration
- BFTS struggles to converge, starting from all-0 parameters.
- Nelder-Mead works without derivative

- None of the attempts introduce sparsity!
- Using pretrained parameters (params_warm), all attempted solvers converge successfully

In [ ]:
def train_sgd(obj, grad, X, y, 
              tol = 1e-8, 
              max_iter = 10000,
              exp_dec1 = 0.9,
              exp_dec2 = 0.999,
              eta = 0.01,
              epsilon = 1e-8,
              scaling = True,
              lamb = 0.9):

    # weights and data params
    n_classes = np.unique(y).size
    params = np.zeros(X.shape[0] + n_classes)
    
    # adam
    m_dw, v_dw = 0, 0
    m_db, v_db = 0, 0

    # iteration tracking
    dloss = 1
    cur_iter = 1
    losses = []
    while True:
        if (dloss < tol):
            print("Converged")
            break
        if (cur_iter > max_iter):
            print("Max iterations")
            break

        loss = - obj(params, X, y, n_classes, lamb, scaling)
        g = grad(params, X, y, n_classes, lamb, scaling)

        # split into w, b
        #w = params[:-n_classes]
        #b = params[-n_classes:]
        #dw = g[:-n_classes]
        #db = g[-n_classes:]
        
        # ADAM:
        # momentum
        m_dw = exp_dec1 * m_dw + (1 - exp_dec1) * g
        #m_db = exp_dec1 * m_db + (1 - exp_dec1) * db
        # rms
        v_dw = exp_dec2 * v_dw + (1 - exp_dec2) * (g**2)
        #v_db = exp_dec2 * v_db + (1 - exp_dec2) * (db)
        # bias correction
        m_dw_corr = m_dw / (1 - exp_dec1**cur_iter)
        #m_db_corr = m_db / (1 - exp_dec1**cur_iter)
        v_dw_corr = v_dw / (1 - exp_dec2**cur_iter)
        #v_db_corr = v_db / (1 - exp_dec2**cur_iter)

        # update weights
        params = params - eta * (m_dw_corr / (np.sqrt(v_dw_corr) + epsilon))
        #params[:-n_classes] = w - eta * (m_dw_corr / (np.sqrt(v_dw_corr) + epsilon))
        #params[-n_classes:] = b - eta * (m_db_corr / (np.sqrt(v_db_corr) + epsilon))
        
        losses.append(loss)
        if len(losses) > 1:
            dloss = np.abs(losses[-2] - losses[-1])
        else:
            dloss = tol + 1
        
        cur_iter += 1

    print("iterations", cur_iter)
    print("Final loss", loss)
    print("Final grad", g)
    print("Final weights", params)
    

    return (w, losses)

In [ ]:
w, losses = train_sgd(objective, objective_grad, ac_X_train.T, ac_y_train, scaling=True, max_iter=10000, lamb=10, eta=0.1, tol=1e-6)

In [ ]:
plt.plot(np.arange(len(losses)), [-1* l for l in losses])

In [ ]:
train_sgd(objective, objective_grad, ac_X_train.T, ac_y_train, scaling=True, lamb=10)

## Use Jax autograd

In [ ]:
from jax import grad, jit, vmap
from jax import numpy as jnp

In [ ]:
LMBD = 0.5
BIG = 1e10
X = ac_X_train
y = ac_y_train
k = jnp.unique(ac_y_train).size

def jphi(beta, thresholds):
    phi = 1 / (1 + jnp.exp(jnp.dot(beta.T, X) - thresholds))
    return phi

def jl1_regularizer(beta):
    return LMBD * jnp.sum(jnp.abs(beta))

# PROBLEM: Is the objective minimized or maximized?? -> Scipy MINIMIZES
# log likelihood with regularization as objective function
def jobjective(params):
    beta = params[:-k]
    theta = params[-k:]
    thresholds = jnp.array([theta[i] for i in y])
    thresholds_m1 = jnp.array([theta[max(0, i-1)] for i in y]) 

    # fit term
    # PROBLEM: What if diff is negative? -> Log has issues, but value may be meaningful
    idx = (thresholds != 0)
    diff = jnp.zeros_like(thresholds)
    phi_i = jphi(beta, thresholds)
    phi_im1 = jphi(beta, thresholds_m1)
    diff.at[idx].set((phi_i - phi_im1)[idx])
    diff.at[~idx].set(phi_i[~idx])

    loss = jnp.log(diff)
    loss = jnp.nan_to_num(jnp.log(diff), nan=0)
    loss = jnp.sum(loss)

    # regularization term
    loss -= jl1_regularizer(beta)
    
    # scipy MINIMIZES the loss
    loss *= -1

    #print("avg loss:", loss.mean())
    return loss


In [ ]:
from scipy.optimize import minimize

LMBD = 0.01
BIG = 1e10
X = ac_X_train.T
y = ac_y_train
k = jnp.unique(ac_y_train).size
params = np.zeros(ac_X_train.shape[1] + k)

params_ones = np.ones(ac_X_train.shape[1] + k)

params_warm_start = [0.47077975,  0.        ,  0.37275445,  0.        ,  0.        ,
        -0.2718176 ,  0.01834662,  0.27956828,  0.        ,  0.00906079,
         0.        ,  0.22041753, -0.1378634 ,  0.18726233, -0.08640451,
         0.08268969, -0.26427331,  0.        , -0.28873376, -0.24587408,
         0.        , -0.07656282, -0.02134246,  0.59662556,  0.05288458,
         0.        ,  1.71402649,  0.10353958,  0.        , 0,  -0.48783945,
        -0.62012817, -1.75287372, -2.09708938]



objective_jaxder = grad(jobjective)
minimize(jobjective, params, jac=objective_jaxder, method="BFGS")